## preparation

In [4]:
import pandas as pd
import numpy as np
import scipy.optimize
import random

In [5]:
fares = pd.read_csv('Fares.csv', sep = ';')
flights = pd.read_csv('Flights.csv', sep = ';')

In [6]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')

In [7]:
history = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_001.csv', sep = ';')
history['id'] = '001'
for i in range(2,101,1):
    if len(str(i)) == 1:
        i = '00' + str(i)
    elif len(str(i)) == 2:
        i = '0' + str(i)
    else:
        i = str(i)
        
        
    hist = pd.read_csv('~\Documents\GitHub\Quant_Marketing_UvA\Ass_3\History\Bookings_' + i + '.csv', sep = ';')
    hist['id'] = i
    history = history.append(hist)
    
    
   

In [8]:
def get_demands(fare_id, history):
    filt_history = history.loc[history.FARE_ID == fare_id,]
    demands = filt_history['id'].value_counts()
    return demands

def get_capacity(flight_id):
    return flights.loc[flights.FLIGHT_ID == flight_id]['CAPACITY']
    
def get_price(fare_id):
    return fares.loc[fares.FARE_ID == fare_id]['PRICE']

def get_flights(fare_id):
    dep_air  = int(fares.loc[fares.FARE_ID == fare_id]['DEPARTURE_AIRPORT'])
    dest_air = int(fares.loc[fares.FARE_ID == fare_id]['DESTINATION_AIRPORT'])
   
    if dep_air == 1 or dest_air == 1:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air) &
                           (flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]
    else:
        return [flights.loc[(flights.DEPARTURE_AIRPORT == dep_air)]['FLIGHT_ID'], 
                flights.loc[(flights.DESTINATION_AIRPORT == dest_air)]['FLIGHT_ID']]




    

In [9]:
flight_fare_df = pd.DataFrame(index=range(flights.shape[0]),columns=range(fares.shape[0]), data = 0)
for i in range(fares.shape[0]):
    used_flights = get_flights(i+1)
    try: 
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        flight_fare_df.iloc[int(used_flights[1]) - 1, i] = 1
    except:
        flight_fare_df.iloc[int(used_flights[0]) - 1, i] = 1
        

## optimization

In [69]:
print(fares.columns, fares.shape)
print(flights.columns, flights.shape)
print(history.columns, history.shape)

Index(['FARE_ID', 'DEPARTURE_AIRPORT', 'DESTINATION_AIRPORT', 'CLASS',
       'PRICE'],
      dtype='object') (144, 5)
Index(['FLIGHT_ID', 'DEPARTURE_AIRPORT', 'DESTINATION_AIRPORT', 'CAPACITY'], dtype='object') (16, 4)
Index(['TIME', 'FARE_ID', 'id'], dtype='object') (24120, 3)


In [10]:
def kvadr_x(x):
    return (x ** 2 - 3 * x + 8) 

In [11]:
cons = ( {'type': 'ineq',
            'fun' : lambda x: np.array([-x, 0])})

In [12]:
x0 = np.array([100])
scipy.optimize.minimize(kvadr_x, x0,  method='SLSQP', constraints = cons)

     fun: 7.999999999999997
     jac: array([-3.])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([8.8817842e-16])

In [ ]:
#problem je v constraints nekde
def profit(x):
    return -(np.array(fares['PRICE']) @ x)

capacity = flights['CAPACITY']
capacity = np.reshape(capacity, (16, 1))
x0 = np.zeros((fares.shape[0], ))

print(capacity.shape)
print(np.reshape(np.array(flight_fare_df) @ x0,(16)).shape)
print(x0.shape)

cons = ({'type' : 'ineq',
         'fun' : lambda x: np.array([capacity, np.reshape(np.array(flight_fare_df) @ x,(16, 1))])})
         
scipy.optimize.minimize(profit, x0,  method='SLSQP', constraints = cons)
         



In [99]:
exp_demand = np.array(history['FARE_ID'].value_counts().sort_index())

In [100]:
exp_demand

array([338, 145, 176,  82, 416, 132,   3,   1, 364, 178, 251,  78, 289,
       123, 323, 132,   4,   3,  91,  35, 514, 232, 210, 101, 205,  91,
       483, 206, 320, 118,  78,  17, 514, 231, 193,  73, 197,  78, 320,
        83, 387,  95, 250,  99, 244,  85, 360, 115, 120,  48, 301, 102,
       326,  94, 381, 154, 129,  56, 131,  37, 425, 131,  89,  35,   5,
         1, 103,  34, 389, 116, 268,  86,  83,  31, 420, 170, 365, 128,
        50,  21,  13,   6,  48,  10, 366, 155, 315, 118, 191,  69, 173,
        50, 413, 139, 211,  80, 380, 120,  26,   8, 486, 156,  35,  11,
       238,  74,  32,   6, 317, 130, 488, 174, 132,  49, 190,  52, 162,
        71, 250, 102, 239,  99, 334, 138, 427, 127, 303, 108,  56,  14,
       179,  53, 392,  93, 117,  35,  92,  32, 208,  57, 369, 155, 390,
       165], dtype=int64)

In [101]:
bounds = [(0, exp_demand[i]) for i in range(len(exp_demand))]

In [110]:
allocations = scipy.optimize.linprog(c= - np.array(fares['PRICE']), A_ub=np.array(flight_fare_df), 
                                     b_ub=capacity, bounds=bounds)

In [111]:
np.sum(allocations.x)

182.0

In [112]:
shadow_prices = scipy.optimize.linprog(c=capacity.reshape((16,)), A_ub= - np.array(flight_fare_df).T,
                                       b_ub= - np.array(fares['PRICE']).T)

In [113]:
shadow_prices

     fun: 87336.0
 message: 'Optimization terminated successfully.'
     nit: 232
   slack: array([168.,   0., 238.,   0., 469.,   0., 442.,   8., 427.,   0., 133.,
         0., 147.,   0., 329.,   0., 168.,   0., 411.,  40., 681., 352.,
       640., 248., 624., 232., 310.,  72., 316.,   8., 497.,   0., 238.,
         0., 411.,  40., 726., 152., 680.,   8., 665.,   0., 405., 272.,
       421., 288., 604., 296., 469.,   0., 681., 352., 726., 152., 947.,
       296., 931., 280., 622., 160., 621.,  40., 798.,   0., 442.,   8.,
       640., 248., 680.,   8., 947., 296., 990., 976., 574.,   0., 593.,
        40., 794., 192., 427.,   0., 624., 232., 665.,   0., 931., 280.,
       990., 976., 560.,   0., 579.,  40., 780., 192., 133.,   0., 310.,
        72., 405., 272., 622., 160., 574.,   0., 560.,   0., 302., 176.,
       477., 120., 147.,   0., 316.,   8., 421., 288., 621.,  40., 593.,
        40., 579.,  40., 302., 176., 512., 288., 329.,   0., 497.,   0.,
       604., 296., 798.,   0., 7

In [65]:
np.array(fares['PRICE'])

array([ 24, 192,  34, 272,  67, 536,  62, 496,  61, 488,  19, 152,  21,
       168,  47, 376,  24, 192,  53, 424,  47, 376,  56, 448,  56, 448,
        34, 272,  44, 352,  71, 568,  34, 272,  53, 424,  82, 656,  96,
       768,  95, 760,  19, 152,  19, 152,  44, 352,  67, 536,  47, 376,
        82, 656,  93, 744,  93, 744,  66, 528,  83, 664, 114, 912,  62,
       496,  56, 448,  96, 768,  93, 744,   2,  16,  82, 656,  79, 632,
        86, 688,  61, 488,  56, 448,  95, 760,  93, 744,   2,  16,  80,
       640,  77, 616,  84, 672,  19, 152,  34, 272,  19, 152,  66, 528,
        82, 656,  80, 640,  18, 144,  51, 408,  21, 168,  44, 352,  19,
       152,  83, 664,  79, 632,  77, 616,  18, 144,  32, 256,  47, 376,
        71, 568,  44, 352, 114, 912,  86, 688,  84, 672,  51, 408,  32,
       256], dtype=int64)